In [65]:
import cv2
import cv2 as cv
import numpy as np
import string
import os
from copy import deepcopy


In [66]:
RESIZE_CONSTANT=(80,100)

In [67]:
def show_image(title,image):
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def invert_image(image):
    return cv.bitwise_not(image)

img = cv.imread('./svg_formated/02yy.png')

def preprocess_image(img):
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    
    thresh_min = cv.threshold(invert_image(gray), 1, 255, cv.THRESH_BINARY)[1]
    
    thresh_min = invert_image(thresh_min)
    thresh = cv.threshold(gray,1, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)[1]
    
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3,8))
    morph_img = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)
    
    kernel = np.ones((2, 2), np.uint8)
    morph_img = cv.erode(morph_img, kernel)
    
    kernel = np.ones((5, 5), np.uint8)
    morph_img = cv.dilate(morph_img, kernel)

    image1 = morph_img.tolist()
    image2 = thresh_min.tolist()
    
    my_img = [[0 for i in range(len(x))] for x in image1]
    
    for i in range(len(image1)):
        for j in range(len(image1[0])):
            my_img[i][j] = max(0,image1[i][j] - image2[i][j])
    
    
    my_img = np.array(my_img)
    my_img = my_img.astype(np.uint8)
    
    return my_img

def get_data(directory):
    result = []
    for filename in os.listdir(directory):
        f = os.path.join(directory,filename)
        result.append(f)
    result.sort()
    return result


def contours_to_rectangles(contours):
    letter_image_regions = []
    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        if w*h < 200:
            continue
        
        letter_image_regions.append((x,y,w,h))
        
    return letter_image_regions

image_without_noise = preprocess_image(img)
show_image("img",image_without_noise)

In [68]:
img = cv.imread('./svg_formated/02yy.png')
image_without_noise = preprocess_image(img)

contours = cv2.findContours(image_without_noise.copy(),cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)[0]
letter_image_regions = contours_to_rectangles(contours)


for rect in letter_image_regions:
    x1,y1,x2,y2 = rect
    cv.rectangle(img,rect,(123,0,0),1)
show_image("img",img)

    

In [69]:
#extract letters
ch_path = './let_formated'
ch_files = get_data(ch_path)
crop_path = './let_cropped/'

for ch_file in ch_files:
    ch_img = cv.imread(ch_file)
    
    ch_preprocessed = preprocess_image(ch_img)
    
    contours = cv.findContours(ch_preprocessed.copy(),cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)[0]
    
    ch_image_regions = contours_to_rectangles(contours)
    
    for rect in ch_image_regions:
        x,y,w,h = rect
        ch_cropped = ch_img[y-5:y+5+h,x-5:x+5+w]
        ch_cropped = cv.resize(ch_cropped,RESIZE_CONSTANT)
        cv.imwrite(crop_path+ch_file[-5:], ch_cropped)


In [70]:
def classify_letter(patch):
    
    def transform_image(img):
        gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

        thresh = cv.threshold(gray,1, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)[1]

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (4,8))
        morph_img = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)
        
        kernel = np.ones((3, 3), np.uint8)
        morph_img = cv.erode(morph_img, kernel)
        
        return morph_img
    

    maxi = -np.inf
    rez = ''
    
    
    patch_sharpened = transform_image(patch)
    
    for filename in os.listdir(crop_path):
        if filename[-3:] == 'png':
            lt = cv.imread(crop_path+filename)
            lt_sharpened = transform_image(lt)
            corr = cv.matchTemplate(patch_sharpened,lt_sharpened[5:-5,5:-5],cv.TM_CCOEFF_NORMED)
            corr = np.max(corr)
            if corr > maxi:
                maxi = corr
                rez = filename[:-4]
    return rez

#classify captchas
captcha_path = './svg_formated'
captcha_files = get_data(captcha_path)

def cmp(x):
    return x[0]

true_labels = []
predicted_labels = []

for captcha_file in captcha_files:
    
    true_label = captcha_file[-8:-4]
    true_labels.append(true_label)
        
    captcha = cv.imread(captcha_file)
    
    captcha_preprocessed = preprocess_image(captcha)
    
    contours = cv.findContours(captcha_preprocessed.copy(),cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)[0]
    
    captcha_image_regions = contours_to_rectangles(contours)
    captcha_image_regions.sort(key=cmp)
    
    label = ''
    for rect in captcha_image_regions:
        x,y,w,h = rect
        character = captcha[y-5:y+5+h,x-5:x+5+w]
        
        character = cv.resize(character,RESIZE_CONSTANT)
        
        x = classify_letter(character)
        label+=x
    
    predicted_labels.append(label)


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [71]:
print(predicted_labels)
print(true_labels)

['e2yy', 'eCqV', '2DAY', '2s1V', '3aaa', '44', '4fvb', '4w1E', '5FTx', '6Q4w', '6zhm', 'smIw', '9Xux', '9nGb', 'BQX', 'Cm8', 'DkRN', 'DqlY', 'EHQ', 'FPhy', 'Fdej', 'FlfK', 'G8ZN', 'mFB', 'GdF9', 'H683', 'NeJ2', 'Npyu', 'MvnJ', 'lJu6', 'ITBV', 'lX8R', 'leDr', 'JkQ', 'HR', 'KSbr', 'K5k', 'Mfje', 'KJb', 'NLGE', 'eV', 'NcT3', 'NpiV', 'O2eU', 'ONCy', 'Ohmf', 'P4hh']
['02yy', '0CqV', '2DAY', '2o1v', '3aaa', '4HMB', '4fvb', '4w1E', '5FTx', '6Q4w', '6zhm', '8miw', '9Xux', '9nGb', 'BZkX', 'Cxw8', 'DkRN', 'DqlY', 'EHXG', 'FPhy', 'Fdej', 'FlfK', 'G8ZN', 'GHFB', 'GdF9', 'H683', 'HeJ2', 'Hpyu', 'HvnJ', 'IJu6', 'ITBV', 'IX8R', 'IeDr', 'JkEV', 'KNYR', 'KSbr', 'M45k', 'Mfjh', 'MnJb', 'NLGE', 'NMzV', 'Ncr3', 'NpiV', 'O2eU', 'ONCy', 'Ohmf', 'P4hh', 'PE1i']


In [72]:
print(sum([1 for i in range(len(predicted_labels)) if predicted_labels[i] == true_labels[i]]) / len(predicted_labels))

0.5319148936170213
